In [4]:
import shutil
import os, shutil
from skimage.exposure import match_histograms
import numpy as np
import csv

%run ./variables.ipynb
%run ./utils.ipynb
%run ../utils/data_utils.ipynb
%run ../utils/image_utils.ipynb

TEST_SIZE = 0.1
norm = False
crossval=False

In [5]:
# Get taxa list for filtering
if not FILTER_PATH is None:
    selected_taxa = get_taxa_list(FILTER_PATH)
    print("Filter:", len(selected_taxa), "taxa to select!")

# Loading reference image for histogram matching and saving ref img
ref = cv2.imread("/mnt/nvme-storage/pfauregi/datasets/atlas/ref_img.png", cv2.IMREAD_GRAYSCALE)
cv2.imwrite(os.path.join(SAVED_MODELS_ROOT, "ref_img.png"), ref)

# Fetching files
taxa_dict = {}
for path in DATASET:
    print("Processing:",path)
    for taxon in os.listdir(path):
        if (FILTER_PATH is None) or (taxon in selected_taxa):
            dir_path = os.path.join(path, taxon)
            files = [f for f in os.listdir(dir_path) if isfile(join(dir_path, f))]
            for file in files:
                split = file.split(".")
                if (len(split)>1 and split[1] in ["png", "tiff", "tif", "TIF"]):
                    file_root = file.split(".")[0]
                    source_file = os.path.join(dir_path, file)
                    target_file = os.path.join(taxon, file_root+".png")
                    taxa_dict.setdefault(taxon, []).append({"source": source_file, "target": target_file})

Filter: 80 taxa to select!
Processing: /mnt/nvme-storage/pfauregi/datasets/Aqualitas/


In [6]:
# Filtering
X, y = [], []
eliminated_taxa = {}
for taxon in taxa_dict:
    files_tmp = taxa_dict[taxon]
    if len(files_tmp)>=RANGE[0]:
        if len(files_tmp)>=RANGE[1]: files_tmp = np.random.permutation(files_tmp)[0:RANGE[1]]
        X.extend(files_tmp)
        y.extend([taxon]*len(files_tmp))
    else:
        eliminated_taxa.setdefault(taxon, None)
eliminated_taxa = list(eliminated_taxa.keys())
print(len(X) ,"images detected belonging to", len(np.unique(y)), "classes found in",len(DATASET),"folder!")
print("Eliminated taxon (unsufficient number of images):", eliminated_taxa)

# Building dataset
check_dirs(DATASET_PATH)
delete_all_files_in_folder(DATASET_PATH)
a = display(str(0)+"/"+str(len(X)),display_id=True)
for i in range(len(X)):
    a.update(str(i+1)+"/"+str(len(X)))
    taxon = y[i]
    source_file = X[i]["source"]
    target_file = os.path.join(DATASET_PATH, X[i]["target"])
    check_dirs(target_file)
    img = cv2.imread(source_file, cv2.IMREAD_GRAYSCALE)
    if norm: img = match_histograms(img, ref, multichannel=False).astype("uint8")
    img = convert_to_square(img, new_size=256)
    cv2.imwrite(target_file, img)
        
# Save dataset infos
csv_path = os.path.join(DATASET_PATH, 'dataset_infos.csv')
f = open(csv_path, 'w')
n_taxa = []
with f:
    writer = csv.writer(f)
    writer.writerow(["taxon", "n_images"])
    for taxon in taxa_dict:
        if not taxon in eliminated_taxa:
            writer.writerow([taxon, len(taxa_dict[taxon])])
            n_taxa.append(len(taxa_dict[taxon]))
print("Dataset info saved at:",csv_path)
print(np.mean(n_taxa),"+-",np.std(n_taxa),"images per taxon!")
print("Finished !")

8668 images detected belonging to 80 classes found in 1 folder!
Eliminated taxon (unsufficient number of images): []
Deleting all files in /mnt/nvme-storage/pfauregi/training/thumbails/aqualitas/dataset/


'8668/8668'

Dataset info saved at: /mnt/nvme-storage/pfauregi/training/thumbails/aqualitas/dataset/dataset_infos.csv
108.35 +- 46.95239610499127 images per taxon!
Finished !
